In [9]:
 import findspark
 findspark.init()

In [10]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext,SparkSession
from pyspark.sql.types import *
from pyspark.rdd import RDD
from branca.colormap import linear
import folium
import folium.plugins
from folium import features
import ipywidgets as widgets
import ipywidgets as wg
from ipywidgets import Button, HBox, VBox
from IPython.display import display,clear_output,IFrame
from itertools import islice
from googlesearch import *
import numpy as np
import webbrowser
import json
import os
import re
import csv
import dask.dataframe

In [11]:
conf = SparkConf().setAppName('USA-app').setMaster('local')
sc = SparkContext.getOrCreate(conf=conf)
sqlc = SQLContext(sc)

spark = SparkSession \
    .builder \
    .appName("USA accidents mining") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [12]:
info = json.load(open("json/accidents-info.json"))

accidents = spark.read.csv("csv/US_Accidents_May19.csv",header=True,sep = ',');
accidents.createOrReplaceTempView("accidents")
result = spark.sql("select * from accidents ")

def tabellen():
    def f(x):    
        if x == 1:
            return result.select("ID","Source","TMC","Severity","Start_Time","End_Time","Start_Lat","Start_Lng","Distance(mi)").show(20)
        elif x == 2:
            return result.select("ID","Description").show(20,False)  
        elif x == 3:
            return result.select("ID","Number","Street","Side","City","County","State","Zipcode","Country","Timezone").show(21)
        elif x == 4:
            return result.select("ID","Airport_code","Weather_Timestamp","Temperature(F)","Wind_Chill(F)","Humidity(%)","Pressure(in)").show(20)
        elif x == 5:
            return result.select("ID","Visibility(mi)","Wind_Direction","Wind_Speed(mph)","Precipitation(in)","Weather_Condition").show(20)
        elif x == 6:
            return result.select("ID","Amenity","Bump","Crossing","Give_Way","Junction","No_Exit","Railway","Roundabout").show(20)
        elif x == 7:
            return result.select("ID","Station","Stop","Traffic_Calming","Traffic_Signal","Turning_Loop").show(20)
        elif x == 8:
            return result.select("ID","Sunrise_Sunset","Civil_Twilight","Nautical_Twilight","Astronomical_Twilight").show(20)
        else:
            return print("error in selectie van tabel")

    wg.interact(f, x=wg.IntSlider(min=1,max=8,step=1));

AttributeError: 'DataFrame' object has no attribute 'tr5'

In [13]:
text = widgets.Text()
code_text = widgets.Text()

def handle_submit(sender):
    for i in info:
        if text.value == i or text.value == str(i).lower(): 
            print(info[i]["explanation"])
            result.select(i).show(10)
    if text.value == "TMC" or text.value == "tmc" :
        [print("code: " + i,end=' ') for i in info["TMC"]["codes"]] 
        display(code_text)

In [14]:
def handle_submit2(sender):
    for i in info["TMC"]["codes"]:
        if code_text.value == i:
            print(info["TMC"]["codes"][i])

In [86]:
#avi
# returns columns and number of items as json

def getJsonFromAccidentFile(take, *cols):
    if cols[0] == -1:
        sqllatlng = result.select("*").dropna().rdd.map(lambda r: r).take(take)
    elif cols[0] == -2:
        whereState = cols[1]
        colArr = []
        for col in cols:
            colArr.append(col)
        colArr.pop(0)
        colArr.pop(0)
        #colsArray = np.asarray(cols)
        sqllatlng = result.select(colArr).where(whereState).dropna().rdd.map(lambda r: r).take(take)
    else:
        colArr = []
        for col in cols:
            colArr.append(col)
        #colsArray = np.asarray(cols)
        sqllatlng = result.select(colArr).dropna().rdd.map(lambda r: r).take(take)
    
    jsonlatlng = json.dumps(sqllatlng)
    load = json.loads(jsonlatlng)
    return load

In [16]:
#avi

def getJsonFromHolidaysFile(take, all, *cols):
    accidents = spark.read.csv("csv/usholidays.csv",header=True,sep = ',');
    accidents.createOrReplaceTempView("holidays")
    holidaysTable = spark.sql("select * from holidays ")

    if(all):
        sqllatlng = holidaysTable.select().dropna().rdd.map(lambda r: r).take(take)
    else:
        colArr = []
        for col in cols:
            colArr.append(col)
        sqllatlng = holidaysTable.select(colArr).dropna().rdd.map(lambda r: r).take(take)
    jsonlatlng = json.dumps(sqllatlng)
    load = json.loads(jsonlatlng)
    return load


In [136]:
#avi
#get accidant for holidays
holidays = getJsonFromHolidaysFile(1000, False, "date", "holiday")
holidays = list(filter( lambda x: int(x[0][0:4]) >= 2016, holidays))


# from datetime import datetime 

all = getJsonFromAccidentFile(1000000, "Start_Time")
# strDate =
for holiday in holidays:
    dateCheck = holiday[0]
    # whereState = "'2016-02-08 05:46:00' == Start_Time"

    print(holiday[1] + ", \t"+ str(sum(dateCheck in y[0] for y in all)))


# oness = list(filter( lambda x: dateCheck in x[0], all))
# # holidays
# oness.count()


# datetime_object = datetime.strptime(strDate, '%Y-%m-%d %H:%M:%S')


# sqllatlng = result.select("ID").where('ID=="A-1"').dropna().rdd.map(lambda r: r).take(100)
# jsonlatlng = json.dumps(sqllatlng)
# load = json.loads(jsonlatlng)
# load


New Year's Day, 	0
Birthday of Martin Luther King, Jr., 	0
Washington's Birthday, 	27
Memorial Day, 	326
Independence Day, 	863
Labor Day, 	987
Columbus Day, 	873
Veterans Day, 	1710
Thanksgiving Day, 	1110
Christmas Day, 	423
New Year's Day, 	656
Birthday of Martin Luther King, Jr., 	1251
Washington's Birthday, 	1312
Memorial Day, 	0
Independence Day, 	739
Labor Day, 	0
Columbus Day, 	0
Veterans Day, 	0
Thanksgiving Day, 	0
Christmas Day, 	0
New Year's Day, 	0
Birthday of Martin Luther King, Jr., 	0
Washington's Birthday, 	0
Memorial Day, 	0
Independence Day, 	4
Labor Day, 	1319
Columbus Day, 	2543
Veterans Day, 	2674
Thanksgiving Day, 	1492
Christmas Day, 	482
New Year's Day, 	792
Birthday of Martin Luther King, Jr., 	1953
Washington's Birthday, 	2164
Memorial Day, 	0
Independence Day, 	0
Labor Day, 	0
Columbus Day, 	0
Veterans Day, 	0
Thanksgiving Day, 	0
Christmas Day, 	0
New Year's Day, 	0
Birthday of Martin Luther King, Jr., 	0
Washington's Birthday, 	0
Memorial Day, 	0
Independe

In [8]:
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

tab = widgets.Tab(children = [out1, out2,out3])
tab.set_title(0, 'tables')
tab.set_title(1, 'search info header')
tab.set_title(2, 'google search')
display(tab)

with out1:
    tabellen()
        
with out2:
    text.on_submit(handle_submit)
    code_text.on_submit(handle_submit2) 
    display(text)
    
with out3:
    googletext = widgets.Text()

    def handle_submit(sender):
        for url in search(googletext.value, tld="co.in", num=1, stop = 1, pause = 2):
            display(IFrame(src="https://www.google.com/search?igu=1&ei=&q=%s" % googletext.value, width='100%', height='500px'))

    googletext.on_submit(handle_submit) 
    display(googletext)
    

In [9]:
def replacenth(string, sub, wanted, n):
    where = [m.start() for m in re.finditer(sub, string)][n - 1]
    before = string[:where]
    after = string[where:]
    after = after.replace(sub, wanted)
    newString = before + after
    print(newString)

In [10]:
# for testing only. Takes Lat and long
load = getJsonFromAccidentFile(1, "ID","Start_Lat","Start_Lng")

nieuwelijst = []


# print(type(load))
for w in load:
    lat = w[1].replace(".","")
    lat = lat[:2] + '.' + lat[5:]
    
    long = w[2].replace(".","")
    long = long[:3] + '.' + long[5:]
    
    plakken = "[" + w[1] + "," + w[2] + "]"
#     print(plakken)
#     replac = plakken.replace(".000.000.000","",100).replace(".999.999.900","",50).replace(".199.999.990","",50).replace(".399.999.990","",80).replace(".699.999.990","",100).replace(".600.000.000","",100).replace(".299.999.990","",100).replace(".500.000.000","",100).replace(".799.999.990","",100).replace("3.97","39.97",100).replace("3.98","39.98",100).replace("-8.","-84.",100).replace(".","",10).replace("00000000","",100)

#     selectie = replac[:3] + '.' + replac[3:13]+  "." + replac[14:]
    print(plakken)
    
#     print(selectie)

#     if '.' not in plakken[0:3]:
#         nieuwelijst = plakken[0:3] + plakken[3:7] +plakken[8:11]
#     if '.' not in plakken[11:12]:
#         if '-' not in plakken[11:12]:
#             nieuwelijst += plakken[11:12]
#         if '.' not in plakken[12:15]:
#             nieuwelijst += plakken[12:15] + plakken[15:19] + plakken[20:24]
#             if '.' not in nieuwelijst[15:50]:
#                 cond = ('-10.' not in nieuwelijst and '-11.' not in nieuwelijst and '-12.' not in nieuwelijst and ',' in nieuwelijst and '#VELD!' not in nieuwelijst and '-12121.' not in nieuwelijst and '-12122.' not in nieuwelijst )
#                 if cond:
#                     n = nieuwelijst.replace("121.",'-121.').replace("122.","-122.").replace("117.","-117.").replace("118.","-118.")
#                     #and '34.09,-11' not in n
#                     if '.' in n[0:4] :
#                         if '34.09,118' not in n:
#                             print(n)     
 

[39.865147,-84.058723]


In [16]:
state_geo = 'json/us-states.json'

sqllatlng = result.select("ID","Start_Lat","Start_Lng").dropna().rdd.map(lambda r: r).take(1000)
jsonlatlng = json.dumps(sqllatlng)
load = json.loads(jsonlatlng)

m = folium.Map(location=[0, 0],zoom_start=2 )
cluster = folium.plugins.MarkerCluster().add_to(m)
tile = ['Mapbox Bright', 'Mapbox Control Room','stamenwatercolor','OpenStreetMap']
tooltip = "click me, bitch"

for t in tile:
    folium.TileLayer(t).add_to(m)

    
nieuwelijst = []
for w in load:
    
    
    folium.Marker(location = [w[1],w[2]], popup='<i>'+ w[0] +'</i>', tooltip=tooltip).add_to(cluster)




folium.LayerControl(collapsed=True).add_to(m)


m

In [17]:
file = "csv/shortacc.csv"

#a = sc.textFile(file).filter(lambda x: 'ID' not  in x).map(lambda x: (x.plit(";")[1],1))
a = sc.textFile(file)
#a.filter(lambda x: 'ID' not  in x).reduceByKey(lambda x, y: x+y).collect()
# print sc.textFile( "users.csv" ) \
#     .map(lambda x: (x.split('|')[2],1) ) \
#     .reduceByKey( lambda x,y:x+y ) \
#     .collect()
    
# sc.stop()

In [ ]:
file = "csv/shortacc.csv"

i=0
with open(file) as fh:
    reader = csv.reader(fh, delimiter=";")
    for row in reader:
        #if i==1:
            #print(row[6])
        #i += 1
        for i in row:
            #print(row)


In [ ]:
file = "csv/shortacc.csv"

with open(file) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';',skipinitialspace=False, quoting=csv.QUOTE_NONE)
    result = []
    for row in reader:
        #print(json.dumps(row))
        #row.get('Start_Lat')
        #res = json.dumps(row)
        #print('accidents',y)
        #print((row['Start_Lat']))
        #print(row['Start_Lng'])
        #result.append(json.dumps(row))
        #print(result["ID"])
        #rint(row)
        rdd = sc.parallelize(reader)
        #rdd.keys().collect()

In [ ]:
file = "csv/shortacc.csv"

with open(file,'rt') as csvfile:
    us = csv.reader(csvfile, delimiter=';',skipinitialspace=False, quoting=csv.QUOTE_NONE)  
    for i in us:
        if i == " ":
            print(i[6:7],i[7:8])

In [ ]:
lat = spark.sql("select Start_Lat  from accidents")
lat.select("*").show(5,False)

In [ ]:
#file = sc.textFile('C:\\sparkProject\\sparkProject1920\\csv\\US_Accidents_May19.csv').map(lambda l:l.split(";"))
#file.take(1)

csv = sc.textFile("csv\\shortacc.csv")
header = csv.first()
csvHeader = csv.filter(lambda l: header not in l)

csvHeader.take(10)

for i in csvHeader.take(10)[52:62]:
    print(i)


start_lat1= csvHeader.take(1)[0][52:62]
start_lat2= csvHeader.take(2)[1][51:73]


In [ ]:
with open('csv\\US_Accidents_May19.csv') as csv_file:
    csv_reader = spark.read.csv("csv/US_Accidents_May19.csv",header=True,sep = ';');
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            print(f'\t{row[0]} works in the {row[1]} department, and was born in {row[2]}.')
            line_count += 1
    print(f'Processed {line_count} lines.')

In [ ]:


#file = 'csv/US_Accidents_May19.csv'


#with open(file,'rt') as csvfile:
 #   for row in islice(csv.reader(csvfile), 100000, None):
  #      print(row) 
    #us = csv.reader(csvfile, delimiter=';')  
    #for i in us:
        #print(i[4:5])
        
        
def file_read_from_head(fname, nlines):
        with open(fname,'rt') as f:
                for i in islice(f, nlines):
                        print(i)
file_read_from_head('csv/US_Accidents_May19.csv',2) 

                    
         

In [ ]:
#file = 'csv/US_Accidents_May19.csv'


with open(file,'rt') as csvfile:
    for row in islice(csv.reader(csvfile), 1048550, None):
        row 
        

In [ ]:
with open(file,'rt') as csvfile:
    us = csv.reader(csvfile, delimiter=';')
    for i in us:
        print(i[6])

In [ ]:
df = sqlc.read.format("com.databricks.spark.csv").option("header","true").option("inferschema","true").load(file)
sqllatlng = df.dropna().rdd.map(lambda r: r).collect()
print(sqllatlng)


In [ ]:
#data = sqlc.read.format("com.databricks.spark.csv").option("header","true").option("inferschema","true").load("countries.csv")
file = "csv/shortacc.csv"
data = sc.textFile(file)
header = data.first()

split_data_zonder_header = data.filter(lambda l: l != header)
print(split_data_zonder_header)
#mappeke = data.map(lambda l: l.split(";"))

In [ ]:
df = spark.read.csv("country_gdb.csv",header=True,sep = ';');

df.createOrReplaceTempView("gdbTable")

query_gdb = "select code,GDP_value from gdbTable"

state_geo = os.path.join('countries.geo.json')



state_data = spark.sql(query_gdb)
state_data.printSchema()
data_df = state_data.withColumn("GDP_value", state_data["GDP_value"].cast(IntegerType()))
data_df.printSchema()



m.choropleth(
    state_geo,
    data=data_df.toPandas(),
    columns=['code', 'GDP_value'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=5,
    line_opacity=0.2,
    legend_name='GDP'
)

In [ ]:
#[{"cc": "Afghanistan_Kabul","coordinates": [34.555347, 69.207489]}]
sqllatlng = result.select("ID","Start_Lat","Start_Lng").dropna().rdd.map(lambda r: r).collect()

jsonlatlng = json.dumps(sqllatlng)
load = json.loads(jsonlatlng)


with open('writeacc.json', 'w') as json_file:
    json.dump(load, json_file)
    

writeacc = json.load(open("writeacc.json"))


m = folium.Map(location=[0, 0],zoom_start=2 )
cluster = folium.plugins.MarkerCluster().add_to(m)
tile = ['Mapbox Bright', 'Mapbox Control Room','stamenwatercolor','OpenStreetMap']
tooltip = "click me, bitch"

for t in tile:
    folium.TileLayer(t).add_to(m)
    


# for i in writeacc:
#     print("[" + i[1] + "," + i[2] + "]")

#     folium.Marker("[" + i[1] + "," + i[2] + "]", popup='<i>'+ str(i) +'</i>', tooltip=tooltip).add_to(cluster)
folium.Marker(location = [39.865147,-84.058723], popup='<i>'+ "weed" +'</i>', tooltip=tooltip).add_to(cluster)



folium.LayerControl(collapsed=True).add_to(m)

m        


# for w in writeacc:
#     plakken = "[" + w[1] + "," + w[2] + "]"
    

    
    
#lijstjson = []

# for i in load:
#     lijstjson =  [{"ID":i[0] }]


# for i in sqllatlng:
#     lijstjson = [{"coordinates":i }]


# lijstjson = [
#     {"coordinates":[]}
# ]

# for i in lijstjson:
#     i['coordinates'] = sqllatlng

#print(lijstjson)


    

In [ ]:
file = "csv/shortacc.csv"

jsonlatlng = json.dumps(sqllatlng)
load = json.loads(jsonlatlng)


    
m = folium.Map(location=[0, 0],zoom_start=2 )
cluster = folium.plugins.MarkerCluster().add_to(m)
tile = ['Mapbox Bright', 'Mapbox Control Room','stamenwatercolor','OpenStreetMap']
tooltip = "click me, bitch"

for t in tile:
    folium.TileLayer(t).add_to(m)

for w in load:
    plakken = "[" + w[1] + "," + w[2] + "]"
    replac = plakken.replace(".000.000.000","",100).replace(".999.999.900","",50).replace(".199.999.990","",50).replace(".399.999.990","",80).replace(".699.999.990","",100).replace(".600.000.000","",100).replace(".299.999.990","",100).replace(".500.000.000","",100).replace(".799.999.990","",100).replace("3.97","39.97",100).replace("3.98","39.98",100).replace("-8.","-84.",100).replace(".","",10)
    grande = replac[:3] + '.' + replac[3:13]+  "." + replac[13:]
    #print(grande)
    sqlacc = {"coordinates": grande}
    
    l = sqlacc["coordinates"][1:10]
    r = sqlacc["coordinates"][11:21]
    #print(r)
    
    folium.Marker(location = [l,r], popup='<i>'+ "weed" +'</i>', tooltip=tooltip).add_to(cluster)



folium.LayerControl(collapsed=True).add_to(m)

m 

In [ ]:
nmbs = json.load(open("json/nmbs-stations.json"))
#print(nmbs)
m = folium.Map(location=[0, 0],zoom_start=2 )
cluster = folium.plugins.MarkerCluster().add_to(m)
tile = ['Mapbox Bright', 'Mapbox Control Room','stamenwatercolor','OpenStreetMap']
tooltip = "click me, bitch"

for t in tile:
    folium.TileLayer(t).add_to(m)


for x in nmbs:
    folium.Marker(location = [x["geo"]['lat'],x["geo"]['lon']], popup='<i>'+ x["name"] +'</i>', tooltip=tooltip).add_to(cluster)
    
folium.LayerControl(collapsed=True).add_to(m)

m        